In [45]:
import pandas as pd

# Load event data (already exported from StatsBomb)
events = pd.read_csv('../data/euro2024_events.csv')

# Extract shots dataframe
shots_euro = events[events['type'] == 'Shot'].copy()
euro_matches = pd.read_csv('../data/euro2024_matches.csv')

matches = euro_matches[['match_id', 'home_team', 'away_team']]
def get_opponent(row):
    if row['team'] == row['home_team']:
        return row['away_team']
    elif row['team'] == row['away_team']:
        return row['home_team']
    return None

shots_euro = shots_euro.merge(matches, on='match_id', how='left')
shots_euro['opponent_team'] = shots_euro.apply(get_opponent, axis=1)

pd.set_option('display.max_columns', None)

In [11]:
import pandas as pd
import ast

fifty_fifties = pd.read_csv('../data/euro2024_50_50s.csv')

# Parse the JSON-like strings (they use single quotes) into real dicts
fifty_fifties['50_50_parsed'] = fifty_fifties['50_50'].apply(
    lambda s: ast.literal_eval(s) if pd.notna(s) and isinstance(s, str) else None
)

# Extract outcome fields for easy filtering
fifty_fifties['outcome'] = fifty_fifties['50_50_parsed'].apply(lambda d: d.get('outcome') if isinstance(d, dict) else None)
fifty_fifties['outcome_id'] = fifty_fifties['outcome'].apply(lambda o: o.get('id') if isinstance(o, dict) else None)
fifty_fifties['outcome_name'] = fifty_fifties['outcome'].apply(lambda o: o.get('name') if isinstance(o, dict) else None)

# Filter successful 50/50s (choose by name or by id depending on your data)
success_names = ['Won', 'Success To Team'] 
success_ids = [108, 147]  # adjust if your CSV uses different ids , 109, 148
unsuccess_names = ['Lost', 'Failure To Team']
unsuccess_ids = [109, 148]  # adjust if your CSV uses different ids

successful = fifty_fifties[
    fifty_fifties['outcome_name'].isin(success_names) |
    fifty_fifties['outcome_id'].isin(success_ids)
]

unsuccessful = fifty_fifties[
    fifty_fifties['outcome_name'].isin(unsuccess_names) |
    fifty_fifties['outcome_id'].isin(unsuccess_ids)
]

print(fifty_fifties['50_50_parsed'].shape)
print(successful.shape)
print(unsuccessful.shape)

print(successful[['50_50_parsed', 'outcome_id', 'outcome_name']].head())
print(unsuccessful[['50_50_parsed', 'outcome_id', 'outcome_name']].head())

# Count occurrences of each outcome
outcome_counts = fifty_fifties['outcome_name'].value_counts()
print(outcome_counts)

(293,)
(114, 28)
(147, 28)
                                        50_50_parsed  outcome_id  \
1  {'outcome': {'id': 3, 'name': 'Success To Team'}}           3   
3  {'outcome': {'id': 3, 'name': 'Success To Team'}}           3   
5  {'outcome': {'id': 3, 'name': 'Success To Team'}}           3   
7              {'outcome': {'id': 4, 'name': 'Won'}}           4   
9  {'outcome': {'id': 3, 'name': 'Success To Team'}}           3   

      outcome_name  
1  Success To Team  
3  Success To Team  
5  Success To Team  
7              Won  
9  Success To Team  
                             50_50_parsed  outcome_id outcome_name
0  {'outcome': {'id': 1, 'name': 'Lost'}}           1         Lost
2  {'outcome': {'id': 1, 'name': 'Lost'}}           1         Lost
4  {'outcome': {'id': 1, 'name': 'Lost'}}           1         Lost
6  {'outcome': {'id': 1, 'name': 'Lost'}}           1         Lost
8  {'outcome': {'id': 1, 'name': 'Lost'}}           1         Lost
outcome_name
Lost                   

In [6]:
fifty_fifties['50_50_parsed']

0                 {'outcome': {'id': 1, 'name': 'Lost'}}
1      {'outcome': {'id': 3, 'name': 'Success To Team'}}
2                 {'outcome': {'id': 1, 'name': 'Lost'}}
3      {'outcome': {'id': 3, 'name': 'Success To Team'}}
4                 {'outcome': {'id': 1, 'name': 'Lost'}}
                             ...                        
288                {'outcome': {'id': 4, 'name': 'Won'}}
289               {'outcome': {'id': 1, 'name': 'Lost'}}
290    {'outcome': {'id': 3, 'name': 'Success To Team'}}
291               {'outcome': {'id': 1, 'name': 'Lost'}}
292                {'outcome': {'id': 4, 'name': 'Won'}}
Name: 50_50_parsed, Length: 293, dtype: object